In [5]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

In [9]:

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [2]:

acc=[]
for i in open('/home/trishalaswain/ClientData/Other/Accounts2.txt','r').readlines():
    acc.append(i.rstrip('\n'))

In [3]:
acc

['87906749',
 '87906750',
 '87929530',
 '87906756',
 '87906753',
 '87906751',
 '87906742',
 '87906740',
 '87992586',
 '87992591',
 '87992581',
 '88065343',
 '87947455',
 '87947451',
 '88072311',
 '87805198',
 '87755756',
 '87906757',
 '87906755',
 '87906754',
 '87906752',
 '88009673',
 '87861183',
 '88009667',
 '87861177',
 '87976165',
 '88009668',
 '87976162',
 '87861188',
 '87976169',
 '88009672',
 '87861191',
 '87739569',
 '87861190',
 '88009671',
 '87891259',
 '87861184',
 '87876721',
 '87891269',
 '87891268',
 '87861186',
 '87876716',
 '87992582',
 '88009669',
 '87992578',
 '87906737',
 '87906748',
 '88009674',
 '86723899',
 '87906741',
 '87497372',
 '87906746',
 '87992589',
 '87906738',
 '87992585',
 '87992575',
 '88009676',
 '87992579',
 '88009670',
 '88009664',
 '87906743',
 '87976164',
 '87976166',
 '87929542',
 '87755748',
 '87976170',
 '87992580',
 '87976161',
 '87992576',
 '88009665',
 '87976172',
 '87976168',
 '87906739',
 '88009675',
 '87861192',
 '87976163',
 '87929538',

In [6]:
df_query = pd.DataFrame()

In [10]:
df_all=[]

# for f_id in type_id:
# query ='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     dm.id as document_id,
#     em.id as encounter_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON dm.account_id = em.account_id
#         AND dm.service_date = em.service_date
#         AND dm.physician_id = em.physician_id
#         AND em.is_active = 1
# WHERE
#     is_physician_document = 1
#         AND dm.is_active = 1
#         and account_number in ({})
#         and dpd.id = (select MAX(id)FROM document_processing_detail
#           WHERE document_id = dm.id)

# ORDER BY dm.created_date DESC;
# '''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:
query='''
SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    dpd.id AS DPD_ID,
    dm.path AS filePath,
    dm.id as document_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    ec.code AS final_code,
    esm.last_coding_date
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON dm.account_id = em.account_id
        INNER JOIN
    encounter_status_map esm ON esm.encounter_id = em.id
		left JOIN
    document_code dc ON dc.document_id = dm.id
        left JOIN
    eandm_code ec ON ec.encounter_id = em.id

        AND dm.service_date = em.service_date
        AND dm.physician_id = em.physician_id
        AND em.is_active = 1
WHERE
    is_physician_document = 1
	    AND dc.code_category = 'EANDM'
        AND dm.is_active = 1
        and account_number in ({})
        and dpd.id = (select MAX(id)FROM document_processing_detail
        WHERE document_id = dm.id)
        ORDER BY dm.created_date DESC;
'''.format(','.join(str(x) for x in acc))
           
df_all.append(pd.read_sql(query, con=db_connection))

KeyboardInterrupt: 

In [ ]:
df_query

NameError: name 'df_query' is not defined

In [43]:
# df_query=pd.concat(df_all)

In [44]:
# df_query

In [45]:
# df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [13]:
# df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id']).first().reset_index()

In [14]:
# df_query=df_query.sort_values(['facility_id', 'type_id'])

In [15]:
# df_query.columns

In [16]:
# print(df_query.groupby('facility_id').size().to_markdown())

In [17]:
# commands

In [18]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             # df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_pcs.xml".format(env, pcs, fac_name,enc_id, acc, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_eandm_request.json".format(env, em_req,fac_name, enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((em_req, '{}/{}/{}_{}_eandm_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        # cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.pdf".format(env,ori_path,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [23]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/trishalaswain/Documents/analysis/data/EPIC_Feb/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [24]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [27]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [ ]:
df_query_generator()

In [28]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

[143]
52


""


In [ ]:
# df_query.columns

In [ ]:
# df_query[df_query.account_number.str.contains('0996')]

In [ ]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()